In [1]:
import numpy as np
import time

from flatland.envs.rail_generators import complex_rail_generator
from flatland.envs.schedule_generators import complex_schedule_generator
from flatland.envs.rail_env import RailEnv
from flatland.utils.rendertools import RenderTool
from flatland.envs.observations import GlobalObsForRailEnv
from flatland.envs.observations import TreeObsForRailEnv
from flatland.envs.rail_generators import complex_rail_generator
from flatland.envs.schedule_generators import complex_schedule_generator
from flatland.envs.rail_env import RailEnv
from flatland.envs.rail_env import GridTransitionMap

from flatland.evaluators.client import FlatlandRemoteClient
from flatland.envs.observations import TreeObsForRailEnv
from flatland.envs.predictions import ShortestPathPredictorForRailEnv
import numpy as np

In [2]:
#####################################################################
# Instantiate a Remote Client
#####################################################################
remote_client = FlatlandRemoteClient()



ConnectionError: Error 61 connecting to 127.0.0.1:6379. Connection refused.

In [3]:
NUMBER_OF_AGENTS = 10

nr_start_goal = 10
nr_extra = 2
min_dist = 8
max_dist = 99999
seed = 0
# seed = np.random.randint(0,1000)

my_rail_generator = complex_rail_generator (
        nr_start_goal,
        nr_extra,
        min_dist,
        max_dist,
        seed
    )

env_test = RailEnv(
    
    # parameters taht control the dimensions of the map.
    width = 30,
    height = 30,
    
    # Rail generator: generates new rail networks on each reset
    # simplest rail generators: 
    #         envs.rail_generators.rail_from_manual_specifications_generator and 
    #         envs.rail_generators.random_rail_generator.
    
    rail_generator = my_rail_generator,
    schedule_generator = complex_schedule_generator(),
    obs_builder_object= GlobalObsForRailEnv(),
    number_of_agents = NUMBER_OF_AGENTS
)

## Need modifications to read actions from cpp output.

In [ ]:
#####################################################################
# Define your custom controller
#
# which can take an observation, and the number of agents and 
# compute the necessary action for this step for all (or even some)
# of the agents
#####################################################################
def my_controller(obs, number_of_agents):
    _action = {}
    for _idx in range(number_of_agents):
        _action[_idx] = np.random.randint(0, 5)
    return _action

In [7]:


#####################################################################
# Instantiate your custom Observation Builder
# 
# You can build your own Observation Builder by following 
# the example here : 
# https://gitlab.aicrowd.com/flatland/flatland/blob/master/flatland/envs/observations.py#L14
#####################################################################
# my_observation_builder = TreeObsForRailEnv(
#                                 max_depth=3,
#                                 predictor=ShortestPathPredictorForRailEnv()
#                             )

my_observation_builder = GlobalObsForRailEnv()

#####################################################################
# Main evaluation loop
#
# This iterates over an arbitrary number of env evaluations
#####################################################################
evaluation_number = 0
while True:

    evaluation_number += 1
    # Switch to a new evaluation environemnt
    # 
    # a remote_client.env_create is similar to instantiating a 
    # RailEnv and then doing a env.reset()
    # hence it returns the first observation from the 
    # env.reset()
    # 
    # You can also pass your custom observation_builder object
    # to allow you to have as much control as you wish 
    # over the observation of your choice.
    observation = remote_client.env_create(
                    obs_builder_object=my_observation_builder
                )
    
    if not observation:
        #
        # If the remote_client returns False on a `env_create` call,
        # then it basically means that your agent has already been 
        # evaluated on all the required evaluation environments,
        # and hence its safe to break out of the main evaluation loop
        break
    
    print("Evaluation Number : {}".format(evaluation_number))

    #####################################################################
    # Access to a local copy of the environment
    # 
    #####################################################################
    # Note: You can access a local copy of the environment 
    # by using : 
    #       remote_client.env 
    # 
    # But please ensure to not make any changes (or perform any action) on 
    # the local copy of the env, as then it will diverge from 
    # the state of the remote copy of the env, and the observations and 
    # rewards, etc will behave unexpectedly
    # 
    # You can however probe the local_env instance to get any information
    # you need from the environment. It is a valid RailEnv instance.
    
#     local_env = remote_client.env
    local_env = env_test
    number_of_agents = len(local_env.agents)

# --------------------------------------------------------------------------------------------------------
    # Obtain the rail map from the local copy of env
    original_rail_map = local_env.rail.grid.copy()
    
    result = decode_map(original_rail_map).convert_ori_rail_map()
    

            
# --------------------------------------------------------------------------------------------------------    
    
    
    
    
    
    # Now we enter into another infinite loop where we 
    # compute the actions for all the individual steps in this episode
    # until the episode is `done`
    # 
    # An episode is considered done when either all the agents have 
    # reached their target destination
    # or when the number of time steps has exceed max_time_steps, which 
    # is defined by : 
    #
    # max_time_steps = int(1.5 * (env.width + env.height))
    #
    while True:
        #####################################################################
        # Evaluation of a single episode
        #
        #####################################################################
        # Compute the action for this step by using the previously 
        # defined controlle
        action = my_controller(observation, number_of_agents)

        # Perform the chosen action on the environment.
        # The action gets applied to both the local and the remote copy 
        # of the environment instance, and the observation is what is 
        # returned by the local copy of the env, and the rewards, and done and info
        # are returned by the remote copy of the env
        observation, all_rewards, done, info = remote_client.env_step(action)
        if done['__all__']:
            print("Reward : ", sum(list(all_rewards.values())))
            #
            # When done['__all__'] == True, then the evaluation of this 
            # particular Env instantiation is complete, and we can break out 
            # of this loop, and move onto the next Env evaluation
            break

print("Evaluation of all environments complete...")
########################################################################
# Submit your Results
# 
# Please do not forget to include this call, as this triggers the 
# final computation of the score statistics, video generation, etc
# and is necesaary to have your submission marked as successfully evaluated
########################################################################
print(remote_client.submit())



NameError: name 'observation' is not defined

### utilities

In [2]:


# straightward implementation, could be optimized
# The node id is in the form (x,y), could be a linear mapping from 
# the most leftupper cornner to the most rightlower corner

# Flatland map encoding scheme, NESW -> N:[NESW], E:[NESW], S[NESW], W[NESW]
# Deadend cell is different, agent faces the opposite to the direction that it can goes. 


class decode_map:
    
    
    def __init__(self,original_map):
        self.original_map = original_map
        self.converted_input = {}

    # Determine next nodes for current_node.
    def case_matching(self, current_node, short_bits):
        
        next_nodes = []
        
        if (int(short_bits[0]) == 1):
            
            next_node = (current_node[0]-1, current_node[1])
#             print("N", next_node)
            
            next_nodes.append(next_node)
        
        if(int(short_bits[1]) == 1):
            
            next_node = (current_node[0], current_node[1]+1)
#             print("E", next_node)
            
            next_nodes.append(next_node)

        
        if(int(short_bits[2]) == 1):
            
            next_node = (current_node[0]+1, current_node[1])
#             print("S", next_node)
            
            next_nodes.append(next_node)

        if(int(short_bits[3]) == 1):
            
            next_node = (current_node[0], current_node[1]-1)
#             print("W", next_node)
            
            next_nodes.append(next_node)
        
        return next_nodes
        
    # First find the possible previous nodes, and call another function to find the possible next nodes.
    def slice_16_bits(self, current_node, bits):
        
        previous_nodes = {}
        
        # Facing North
        
        if (bits[0:4] != "0000"):
            
#             print(bits[0:4])
            next_nodes = self.case_matching(current_node, bits[0:4])
#             print("N, and returned next nodes", next_nodes)
            
            # add one previous node, and what nodes the agent can go from current node.
            previous_nodes[(current_node[0]+1,current_node[1])] = next_nodes
            
#             print("N, and current previous nodes", previous_nodes)
            
        # East
        if (bits[4:8] != "0000"):

#             print(bits[4:8])
            next_nodes = self.case_matching(current_node, bits[4:8])
#             print("E",next_nodes)
            
            previous_nodes[(current_node[0], current_node[1]-1)] = next_nodes
            
#             print("E, and current previous nodes", previous_nodes)
# 

        # South
        if (bits[8:12] != "0000"):
            
#             print(bits[8:12])
            next_nodes = self.case_matching(current_node, bits[8:12])
#             print("S",next_nodes)
            
            previous_nodes[(current_node[0]-1, current_node[1])] = next_nodes
            
#             print("S, and current previous nodes", previous_nodes)
       
        # West
        if (bits[12:16] != "0000"):
            
#             print(bits[12:16])
            next_nodes = self.case_matching(current_node, bits[12:16])
#             print("W",next_nodes)
            
            previous_nodes[(current_node[0], current_node[1]+1)] = next_nodes
            
#             print("W, and current previous nodes", previous_nodes)

        return previous_nodes
            
    def convert_ori_rail_map(self):
        for row in range(0,len(self.original_map)):
            for col in range(0,len(self.original_map[row])):
                
                # Current node: (row,col)
                print("Current node:", (row,col))
                current_node = (row,col)
                # Get previous node, decided by which direction the agent is facing.
                # EXCEPT the deadend nodes.
                
                
#                 print('{0:016b}'.format(self.original_map[row][col]))
                previous_next_nodes = self.slice_16_bits(current_node, '{0:016b}'.format(self.original_map[row][col]))
#                 print(previous_next_nodes.items())
                
                for previous,direction in previous_next_nodes.items():
#                     print("direction", direction)

                    self.converted_input[(current_node, previous)] = direction
#         print(self.converted_input)
        return self.converted_input

In [ ]:
def linearize_loc(env, loc):
    return loc[0]*env.width + loc[1]

In [ ]:
    # idx2node: {index:(current, previous)}
    idx2node = {idx:k for idx, k in enumerate(result.keys())} 

    # node2idx: {(current, previous) : index}
    node2idx = {k:idx for idx, k in idx2node.items()}

    # idx2pos: {index:current}
    idx2pos = {k:v[0] for (k,v) in idx2node.items()}

    edges = []
    for cur, prev in result:
        print (cur, prev, result[(cur, prev)])
        for to in result[(cur, prev)]:
            print()
            print(node2idx[(to, cur)])
            edges.append((node2idx[(cur, prev)], node2idx[(to, cur)]))

    # convert agent start information from the format "start_position, direction = a number"
    # to edge

    start_idx = []
    for agent in local_env.agents:
        print(agent.position)
        print(agent.direction) # direction encoding: {0: North, 1: East, 2: South, 3: West}
        current_pos = agent.position
        if(agent.direction ==0):
            prev_pos = (current_pos[0]+1,current_pos[1])
        elif(agent.direction == 1):
            prev_pos = (current_pos[0],current_pos[1]-1)
        elif(agent.direction == 2):
            prev_pos = (current_pos[0]-1,current_pos[1])
        elif(agent.direction == 3):
            prev_pos = (current_pos[0], current_pos[1]+1)
        start_node = (current_pos,prev_pos)
        index = node2idx[start_node]

        start_idx.append(index)
    #     print(start_node, index)

    print(start_idx)
            
    pos2nodes = dict()
    for _, (cur, prev) in idx2node.items():
        if cur not in pos2nodes:
            pos2nodes[cur] = []
        pos2nodes[cur].append((cur, prev))

    target_idx = []
    for i,agent in enumerate(local_env.agents):
        goal_nodes = pos2nodes[agent.target]
        # print(goal_nodes)
        if len(goal_nodes) > 1:
            print("Goal node is not a deadend")

        target_idx.append(node2idx[goal_nodes[0]])
        
    # map file

    node_file = ""
    edge_file = ""


    node_file += str(len(idx2node)) + "\n"
    for idx,node in idx2node.items():
        node_file += str(linearize_loc(local_env, idx2pos[idx])) + "\n"

    edge_file += str(len(edges)) + "\n"
    for edge in edges:
        edge_file += str(edge[0]) + "," + str(edge[1]) +"\n"

    map_file = node_file + edge_file

    file = open("map.txt", "w") 
    file.write(map_file) 
    file.close() 

    # agent file
    start_goal_pos_file = ""
    start_goal_pos_file += str(len(local_env.agents)) + "\n"
    for i,agent in enumerate(local_env.agents):
        start_goal_pos_file +=  str(start_idx[i]) + "," + str(target_idx[i]) +",1\n"


    file = open("agents.txt", "w") 
    file.write(start_goal_pos_file) 
    file.close()